# Recurrent Neural Networks

The neural networks we have investigated are sometimes called feedforward neural networks. 
These types of neural networks can be limited as the inputs are all processed independently by the network. 
The independence is addressed with recurrent neural networks, where the output of a neuron for a given data point is fed in as an input for the following data point. 
This makes recurrent neural networks uniquely suited for modelling sequential data, such as text, speech and time series. 
As a result, recurrent neural networks have significantly influenced the transformer networks that have led to the revolutionary large language models. 

We will use a simple time series dataset for the electrical production to put together a simple recurrent neural network. 
You can [download the dataset here](../data/electric.csv).
````{margin}
```{note}
Here, we also convert the `'date'` column to a `datetime` object.
```
````

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

timeseries = pd.read_csv('../data/electric.csv')
timeseries['date'] = pd.to_datetime(timeseries['date'])

fig, ax = plt.subplots()
timeseries.plot(x='date', y='elec-prod', ax=ax)
ax.tick_params(axis='x', labelrotation=45)
plt.show()

## Feedback

As mentioned above, the difference between a traditional neural network and a recurrent neural network is that the latter includes the hidden state of the previous data point in the activation function. 
That means that for some activation function, $f$, the traditional network as the equation, 

$$
h_i = f(W_xx_i + b_i),
$$

while for a recurrent network, we add the previous hidden state, 

$$
h_i = f(W_hh_{i-1} + W_xx_i + b).
$$

We implement a simple feedback perceptron with Python below, using a [tanh](https://numpy.org/doc/2.0/reference/generated/numpy.tanh.html) activation function. 

In [ ]:
import numpy as np

class SimpleRNNPerceptron:
    """
    A simple RNN perceptron with a single hidden layer.
    
    :param input_size: The size of the input vector
    :param hidden_size: The size of the hidden layer
    """
    def __init__(self, input_size, hidden_size):
        self.hidden_size = hidden_size

        self.W_x = np.random.randn(hidden_size, input_size) * 0.1
        self.W_h = np.random.randn(hidden_size, hidden_size) * 0.1
        self.b = np.zeros((hidden_size, 1))
        self.h_i = np.zeros((hidden_size, 1)) 

    def step(self, x_i):
        """
        Processes a single step

        :param x_i: The input vector
        :return: The output vector
        """
        x_i = x_i.reshape(-1, 1) 
        self.h_i = np.tanh(np.dot(self.W_x, x_i) + np.dot(self.W_h, self.h_i) + self.b)
        return self.h_i

rnn = SimpleRNNPerceptron(input_size=1, hidden_size=5)

We can run this through a single year of our time series data, where the hidden size is 5. 
The hidden size is the number of neurons in the hidden layer. 

In [ ]:
timeseries_2017 = timeseries['elec-prod'][timeseries['date'].dt.year == 2017]
timeseries_2017 /= timeseries_2017.max()

for i, x_i in enumerate(timeseries_2017):
    h_i = rnn.step(np.array([x_i])) 
    print(f"Step {i+1}: Hidden State = {h_i.ravel()}")

The larger the hidden size, the more capacity the network has to learn complex patterns, which comes with a computational cost. 

## Implementation with `pytorch`

The Python library `pytorch` comes with an implementation of [recurrent neural networks](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). 
We advise you to study the documentation to understand how a recurrent neural network may be implemented using `pytorch`. 
Furthermore, you should appreciate the configurability of this `nn.Module` object. 